<a href="https://colab.research.google.com/github/dh610/ai-intensive2/blob/main/lab3/lab3_TF-IDF_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

%cd drive/MyDrive/ai-intensive2

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: 'drive/MyDrive/ai-intensive2'
/content/drive/MyDrive/ai-intensive2/lab3


In [5]:
!git pull

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 4 (delta 3), reused 0 (delta 0), pack-reused 0 (from 0)
Unpacking objects: 100% (4/4), 3.75 KiB | 7.00 KiB/s, done.
From https://github.com/dh610/ai-intensive2
   94724d5..aeb3c58  main       -> origin/main
Updating 94724d5..aeb3c58
Fast-forward
 lab3/lab3_Word Embedding_assignment.ipynb | 1753 ++++++++++++++++++++++++++-----------------------
 1 file changed, 944 insertions(+), 809 deletions(-)


In [6]:
%cd lab3

[Errno 2] No such file or directory: 'lab3'
/content/drive/MyDrive/ai-intensive2/lab3


# Lab 3 : TF-IDF, Cosine Similarity, and Text Classification

@copyright:
    (c) 2023. iKnow Lab. Ajou Univ., All rights reserved.

M.S. Student: Wansik-Jo (jws5327@ajou.ac.kr)

# For assignment

- Python code의 주석 처리되어있는 부분을 구현하면 됩니다.
- MD 형식의 Cell의 [BLANK] 부분을 채우면 됩니다.
- MD 형식의 Cell의 [ANSWER] 부분 이후에 답을 작성하면 됩니다.
- 조교에게 퀴즈의 답과 함께 코드 실행 결과를 보여준 뒤, BB에 제출 후 가시면 됩니다.


## 목차
1. TF-IDF
    - TF 계산하기
    - IDF 계산하기
    - TF-IDF 계산하기
2. TF-IDF를 이용한 문서 유사도 계산하기
    - 코사인 유사도 계산하기
3. TF-IDF를 이용한 문서 분류하기
    - 문서 분류하기
---

## 1. TF-IDF

### Document Classification dataset

- 이전 실습과 동일한 데이터셋 사용

- URL : https://www.kaggle.com/datasets/jensenbaxter/10dataset-text-document-classification

- Kaggle 10 group dataset
- 각 Document는, 10개의 category 중 하나에 속함
- Categories : business, entertainment, food, graphics, historical, medical, politics, space, sport, technologie
- URL 또는 강의노트에서 제공되는 data를 받아서 사용

In [7]:
import nltk

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [10]:
#dataset load
categories = {'business', 'entertainment', 'food', 'graphics', 'historical', 'medical', 'politics', 'space', 'sport', 'technologie'}

import os

dataset = []
for category in categories:
    for filename in os.listdir('data/' + category):
        with open('data/' + category + '/' + filename, 'r') as f:
            instance = {}
            instance['text'] = f.read()
            instance['category'] = category
            dataset.append(instance)

import random
random.seed(42)
random.shuffle(dataset)
train_data = dataset[:int(len(dataset) * 0.8)]
test_data = dataset[int(len(dataset) * 0.8):]

##

import re
import string

#You can use pre-defined tools in nltk, like Tokenizer, Stopword list, etc.

stopwords = nltk.corpus.stopwords.words('english')
lemm = nltk.stem.WordNetLemmatizer()
stemmer = nltk.stem.PorterStemmer()

def clean_text(instance):
    text = instance['text'].lower()

    text = re.sub(r'[^a-z\s]', '', text)

    text = nltk.word_tokenize(text)
    text = [word for word in text if word not in stopwords]
    text = [lemm.lemmatize(word) for word in text]
    text = [stemmer.stem(word) for word in text]
    text = [word for word in text if word not in string.punctuation]

    instance['tokens'] = text


for instance in train_data:
    """
    Implement text preprocessing here what you think is necessary for the TF-IDF model what ever you want.
    You can just use the code from the previous lab.
    """
    clean_text(instance)

for instance in test_data:
    """
    Same as here.
    """
    clean_text(instance)

### TF

- 각 document $d$에 대해, 문서 내 각 단어 $t$의 TF(Term Frequency)를 계산한다.

- 문서의 단어 수로 나누어 정규화한다.

$$ TF(t) = \frac{\text{term } t \text{'s frequency}}{\text{number of words in document}} $$

In [19]:
def compute_tf(tokens):
    tf_dict = {}

    """
    You need to implement the computation of the term frequency here.
    # tf_dict = {'word1': 0, 'word2': 0, ...}
    for word in tokens:
        tf_dict[word] =

    # normalize the term frequency
    for word, count in tf_dict.items():
        tf_dict[word] =

    """
    for word in tokens:
        if word not in tf_dict:
           tf_dict[word] = tf_dict.get(word, 0) + 1

    for word, count in tf_dict.items():
        tf_dict[word] = count / float(len(tokens))

    return tf_dict

for instance in train_data:
    instance['tf'] = compute_tf(instance['tokens'])

for instance in test_data:
    instance['tf'] = compute_tf(instance['tokens'])

print(train_data[0]['tf'])

{'white': 0.006578947368421052, 'admit': 0.006578947368421052, 'balco': 0.006578947368421052, 'drug': 0.006578947368421052, 'link': 0.006578947368421052, 'ban': 0.006578947368421052, 'american': 0.006578947368421052, 'sprinter': 0.006578947368421052, 'kelli': 0.006578947368421052, 'say': 0.006578947368421052, 'knowingli': 0.006578947368421052, 'took': 0.006578947368421052, 'steroid': 0.006578947368421052, 'given': 0.006578947368421052, 'bay': 0.006578947368421052, 'area': 0.006578947368421052, 'lab': 0.006578947368421052, 'cooper': 0.006578947368421052, 'presid': 0.006578947368421052, 'victor': 0.006578947368421052, 'cont': 0.006578947368421052, 'face': 0.006578947368421052, 'feder': 0.006578947368421052, 'trial': 0.006578947368421052, 'next': 0.006578947368421052, 'year': 0.006578947368421052, 'charg': 0.006578947368421052, 'distribut': 0.006578947368421052, 'tax': 0.006578947368421052, 'evas': 0.006578947368421052, 'said': 0.006578947368421052, 'first': 0.006578947368421052, 'tri': 0

### IDF

- 각 단어 $t$에 대해, IDF(Inverse Document Frequency)를 계산한다.

$$ IDF(t) = log(\frac{\text{total number of document}}{\text{number of document which contains } t}) $$

In [18]:
import math

def compute_idf(documents):
    idf_dict = {}
    N = len(documents)

    """
    you need to implement the computation of the inverse document frequency here.
    # idf_dict = {'word1': 0, 'word2': 0, ...}
    for document in documents:
        for word in set(document['tokens']):
            idf_dict[word] =

    # log normalization
    for word, count in idf_dict.items():
        idf_dict[word] =
    """

    for document in documents:
        for word in set(document['tokens']):
            idf_dict[word] = idf_dict.get(word, 0) + 1

    for word, count in idf_dict.items():
        idf_dict[word] = math.log(N / float(count))

    return idf_dict

idf = compute_idf(train_data)

print(idf['music'])

2.855970331178832


### TF-IDF

- 각 단어 $t$에 대해, TF-IDF를 다음과 같이 계산한다.

$$ TF-IDF(t) = TF(t) \times IDF(t) $$

In [20]:
def compute_tfidf(tf, idf):
    tfidf = {word: tf[word] * idf.get(word, 0) for word in tf}
    return tfidf

# Compute TF-IDF for training data
for instance in train_data:
    instance['tfidf'] = compute_tfidf(instance['tf'], idf)

for instance in test_data:
    instance['tfidf'] = compute_tfidf(instance['tf'], idf)

print(train_data[0]['tfidf'])
print(train_data[1]['tfidf'])

{'white': 0.02117681463729079, 'admit': 0.02077796844113004, 'balco': 0.030297172276237443, 'drug': 0.01893387656511584, 'link': 0.021601410171090605, 'ban': 0.02182444669527272, 'american': 0.015841747425341262, 'sprinter': 0.02280089409736662, 'kelli': 0.023069459850789353, 'say': 0.009120357638946648, 'knowingli': 0.03338930141601201, 'took': 0.015066859032865055, 'steroid': 0.02820208193993129, 'given': 0.014301001454941297, 'bay': 0.02496210506428791, 'area': 0.015315150664331156, 'lab': 0.024268943777065368, 'cooper': 0.02460639966119399, 'presid': 0.017264267875799395, 'victor': 0.02616158899056393, 'cont': 0.03338930141601201, 'face': 0.014017992342548594, 'feder': 0.019081724300984645, 'trial': 0.018240715277893296, 'next': 0.01050668021339173, 'year': 0.005108742036177607, 'charg': 0.0178573671981303, 'distribut': 0.020401926244814586, 'tax': 0.020974369514483202, 'evas': 0.03485735109571076, 'said': 0.004984774353273135, 'first': 0.007573830519344783, 'tri': 0.01205645699834

내 모든 문서 $d$에 대해, 각 단어 $t$의 TF-IDF matrix를 얻을 수 있게 되었다.

## 2. TF-IDF를 이용한 문서 유사도 계산하기


해당 TF-IDF matrix를 이용하여, 문서 간 유사도를 계산할 수 있다.

만약 문서 $d_1$과 문서 $d_2$가 주어졌을 때, 문서 간 유사도를 계산하는 방법은 다음과 같다.

1. 문서 $d_1$과 문서 $d_2$에 대한 TF-IDF matrix를 얻는다.
2. 문서 $d_1$과 문서 $d_2$의 TF-IDF matrix를 이용하여, 코사인 유사도를 계산한다.

### Cosine Similarity

- 두 벡터 $v_1$과 $v_2$에 대해, 코사인 유사도는 다음과 같이 계산한다.

$$ \text{Cosine Similarity} = \frac{v_1 \cdot v_2}{\text{norm of } v_1 \times \text{norm of } v_2} $$

- Document Classification task에서는, 문서 $d_1$과 문서 $d_2$의 TF-IDF matrix를 이용하여, 코사인 유사도를 계산한다.

$$ \text{Cosine Similarity} = \frac{\text{TF-IDF vector 1} \cdot \text{TF-IDF vector 2}}{\text{norm of TF-IDF vector 1} \times \text{norm of TF-IDF vector 2}} $$

In [25]:
def cosine_similarity(vec1, vec2):
    # To do deal with the words that are not in the vocabulary you need to check if the word is in the vocabulary.
    dot_product = sum([vec1[word] * vec2[word] for word in vec1 if word in vec2])

    """
    you need to implement the computation of the magnitude of the vectors here.

    vec1_norm =
    vec2_norm =
    """
    vec1_norm = math.sqrt(sum([value ** 2 for value in vec1.values()]))
    vec2_norm = math.sqrt(sum([value ** 2 for value in vec2.values()]))

    if not vec1_norm or not vec2_norm: # if one of the vectors is empty return 0
        return 0

    cosine_sim = dot_product / (vec1_norm * vec2_norm)

    return cosine_sim

## 3. TF-IDF를 이용한 문서 분류하기

TF-IDF와 Cosine Similarity를 이용하여 classification task를 다음과 같이 진행할 수 있다.

1. Training set의 각 category의 문서들에 대해, TF-IDF를 구해 평균을 계산한다.
2. Inference시, test set의 instance에 대해, TF-IDF를 구한다.
3. Training set의 각 category의 평균 TF-IDF와 test set의 instance의 TF-IDF를 이용하여, 코사인 유사도를 계산한다.
4. 가장 높은 코사인 유사도를 가진 category로 분류한다.

In [23]:
def compute_class_avg_tfidf(data):
    class_avg_tfidf = {}
    class_counts = {}

    """
    you need to implement the computation of the class average TF-IDF here.
    for instance in data:
        category =
        tfidf =

        if category not in class_avg_tfidf:
            ~~
        else :
            ~~

    for category in class_avg_tfidf:
        for word in class_avg_tfidf[category]:
            # normalize the class average TF-IDF

    """
    for instance in data:
        category = instance['category']
        tfidf = instance['tfidf']

        if category not in class_avg_tfidf:
            class_avg_tfidf[category] = tfidf
            class_counts[category] = 1
        else:
            for word in tfidf:
                if word not in class_avg_tfidf[category]:
                    class_avg_tfidf[category][word] = tfidf[word]
                else:
                    class_avg_tfidf[category][word] += tfidf[word]

                class_counts[category] += 1

    for category in class_avg_tfidf:
        for word in class_avg_tfidf[category]:
            class_avg_tfidf[category][word] /= class_counts[category]


    return class_avg_tfidf

class_avg_tfidf = compute_class_avg_tfidf(train_data)

print(class_avg_tfidf['business']['music'])
print(class_avg_tfidf['entertainment']['music'])

7.370942213326038e-07
5.2999650575329006e-05


In [26]:
def classify(instance, class_avg_tfidf):
    instance_tfidf = instance['tfidf']
    category_scores = {category: cosine_similarity(instance_tfidf, class_avg_tfidf[category]) for category in class_avg_tfidf}
    return max(category_scores, key=category_scores.get)

def evaluate(test_data, class_avg_tfidf):
    correct = 0
    for instance in test_data:
        predicted = classify(instance, class_avg_tfidf)
        if predicted == instance['category']:
            correct += 1
    return correct / float(len(test_data))

print(evaluate(test_data, class_avg_tfidf))

0.945
